In [48]:
import tensorflow as tf
import numpy as np
import pandas as pd
import scikeras
import pickle
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.boston_housing.load_data()

In [49]:
X_df = pd.DataFrame(X_train)

In [50]:
X_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,1.23247,0.0,8.14,0.0,0.538,6.142,91.7,3.9769,4.0,307.0,21.0,396.90,18.72
1,0.02177,82.5,2.03,0.0,0.415,7.610,15.7,6.2700,2.0,348.0,14.7,395.38,3.11
2,4.89822,0.0,18.10,0.0,0.631,4.970,100.0,1.3325,24.0,666.0,20.2,375.52,3.26
3,0.03961,0.0,5.19,0.0,0.515,6.037,34.5,5.9853,5.0,224.0,20.2,396.90,8.01
4,3.69311,0.0,18.10,0.0,0.713,6.376,88.4,2.5671,24.0,666.0,20.2,391.43,14.65


In [51]:
def build_model(n_hidden=1, n_neurons=25, optimizer='sgd', learning_rate = pow(10,-5), momentum=0):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=X_train.shape[1:]))
    for i in range(n_hidden):
        model.add(tf.keras.layers.Dense(n_neurons, activation='relu'))
    model.add(tf.keras.layers.Dense(1))
    if optimizer == 'sgd':
        model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate), loss='mse', metrics='mae')
    elif optimizer == 'adam':
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='mse', metrics='mae')
    elif optimizer == 'nesterov':
        model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate, nesterov=True), loss='mse', metrics='mae')
    elif optimizer == 'momentum':
        model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=momentum), loss='mse', metrics='mae')
    return model

In [52]:
import os
root_logdir = os.path.join(os.curdir, "tb_logs")
def get_run_logdir(name, val): 
    import time
    ts = int(time.time())
    run_id = str(ts) + name + str(val) 
    return os.path.join(root_logdir, run_id)

In [53]:
es = tf.keras.callbacks.EarlyStopping(patience=10,
                                      min_delta=1.00)

In [54]:
tf.keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [55]:
learning_rates = [0.000001, 0.00001, 0.0001]
lr_list = []
for lr in learning_rates:
    run_logdir = get_run_logdir('_lr_', lr)
    tensorboard_cb = tf.keras.callbacks.TensorBoard(run_logdir)
    model = build_model(learning_rate=lr)
    model.fit(X_train, y_train, epochs=100, callbacks=[tensorboard_cb, es])
    params = model.evaluate(X_test, y_test)
    lr_list.append((lr, params[0], params[1]))
print(lr_list)

Epoch 1/100
13/13 [==============================] - 1s 3ms/step - loss: 2700.2764 - mae: 31.1550
Epoch 2/100
13/13 [==============================] - 0s 3ms/step - loss: 246.2086 - mae: 11.9515
Epoch 3/100
13/13 [==============================] - 0s 3ms/step - loss: 189.4720 - mae: 10.5609
Epoch 4/100
13/13 [==============================] - 0s 3ms/step - loss: 175.1508 - mae: 10.1026
Epoch 5/100
13/13 [==============================] - 0s 3ms/step - loss: 167.5972 - mae: 9.9639
Epoch 6/100
13/13 [==============================] - 0s 3ms/step - loss: 154.0362 - mae: 9.4851
Epoch 7/100
13/13 [==============================] - 0s 3ms/step - loss: 147.4924 - mae: 9.3873
Epoch 8/100
13/13 [==============================] - 0s 3ms/step - loss: 150.3802 - mae: 9.4128
Epoch 9/100
13/13 [==============================] - 0s 5ms/step - loss: 135.4045 - mae: 8.7718
Epoch 10/100
13/13 [==============================] - 0s 2ms/step - loss: 137.8131 - mae: 8.9101
Epoch 11/100
13/13 [==============

In [56]:
open_file = open("lr.pkl", "wb")
pickle.dump(lr_list, open_file)
open_file.close()

In [57]:
open_file = open("lr.pkl", "rb")
loaded_list = pickle.load(open_file)
open_file.close()
print(loaded_list)

[(1e-06, 77.12567901611328, 6.854006290435791), (1e-05, 65.64254760742188, 6.533296585083008), (0.0001, 206.1436767578125, 11.827668190002441)]


In [58]:
tf.keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [59]:
hl_list = []
for hl in range(4):
    run_logdir = get_run_logdir('_hl_', hl)
    tensorboard_cb = tf.keras.callbacks.TensorBoard(run_logdir)
    model = build_model(n_hidden = hl)
    model.fit(X_train, y_train, epochs=100, callbacks=[tensorboard_cb, es])
    params = model.evaluate(X_test, y_test)
    hl_list.append((hl, params[0], params[1]))
print(hl_list)

Epoch 1/100
13/13 [==============================] - 0s 1ms/step - loss: 24234845961206104064.0000 - mae: 1486527616.0000
Epoch 2/100
13/13 [==============================] - 0s 1ms/step - loss: inf - mae: 2260918888248115200.0000              
Epoch 3/100
13/13 [==============================] - 0s 1ms/step - loss: inf - mae: 4824812911053208938256269312.0000
Epoch 4/100
13/13 [==============================] - 0s 1ms/step - loss: nan - mae: nan                            
Epoch 5/100
13/13 [==============================] - 0s 1ms/step - loss: nan - mae: nan
Epoch 6/100
13/13 [==============================] - 0s 1ms/step - loss: nan - mae: nan
Epoch 7/100
13/13 [==============================] - 0s 1ms/step - loss: nan - mae: nan
Epoch 8/100
13/13 [==============================] - 0s 2ms/step - loss: nan - mae: nan
Epoch 9/100
13/13 [==============================] - 0s 1ms/step - loss: nan - mae: nan
Epoch 10/100
13/13 [==============================] - 0s 1ms/step - loss: nan - m

In [60]:
open_file = open("hl.pkl", "wb")
pickle.dump(hl_list, open_file)
open_file.close()

In [61]:
open_file = open("hl.pkl", "rb")
loaded_list = pickle.load(open_file)
open_file.close()
print(loaded_list)

[(0, nan, nan), (1, 80.50528717041016, 7.194814682006836), (2, 67.43660736083984, 6.457210540771484), (3, 78.75505065917969, 7.431304454803467)]


In [62]:
tf.keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [63]:
nn_list = []
nn = [5, 25, 125]
for neurons in nn:
    run_logdir = get_run_logdir('_nn_', neurons)
    tensorboard_cb = tf.keras.callbacks.TensorBoard(run_logdir)
    model = build_model(n_neurons = neurons)
    model.fit(X_train, y_train, epochs=100, callbacks=[tensorboard_cb, es])
    params = model.evaluate(X_test, y_test)
    nn_list.append((neurons, params[0], params[1]))
print(nn_list)

Epoch 1/100
13/13 [==============================] - 1s 1ms/step - loss: 5503.8062 - mae: 47.5473
Epoch 2/100
13/13 [==============================] - 0s 1ms/step - loss: 585.7496 - mae: 22.3859
Epoch 3/100
13/13 [==============================] - 0s 1ms/step - loss: 585.4879 - mae: 22.3800
Epoch 4/100
13/13 [==============================] - 0s 1ms/step - loss: 585.2284 - mae: 22.3742
Epoch 5/100
13/13 [==============================] - 0s 1ms/step - loss: 584.9681 - mae: 22.3684
Epoch 6/100
13/13 [==============================] - 0s 667us/step - loss: 584.7070 - mae: 22.3626
Epoch 7/100
13/13 [==============================] - 0s 1ms/step - loss: 584.4481 - mae: 22.3568
Epoch 8/100
13/13 [==============================] - 0s 1ms/step - loss: 584.1886 - mae: 22.3510
Epoch 9/100
13/13 [==============================] - 0s 1ms/step - loss: 583.9299 - mae: 22.3452
Epoch 10/100
13/13 [==============================] - 0s 1ms/step - loss: 583.6702 - mae: 22.3394
Epoch 11/100
13/13 [======

In [64]:
open_file = open("nn.pkl", "wb")
pickle.dump(nn_list, open_file)
open_file.close()

In [65]:
open_file = open("nn.pkl", "rb")
loaded_list = pickle.load(open_file)
open_file.close()
print(loaded_list)

[(5, 589.6301879882812, 22.50303077697754), (25, 65.64254760742188, 6.533296585083008), (125, 66.20671844482422, 6.201277732849121)]


In [66]:
tf.keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [67]:
opt_list = []
optimizers = ['sgd', 'adam', 'nesterov', 'momentum']
for opt in optimizers:
    run_logdir = get_run_logdir('_opt_', opt)
    tensorboard_cb = tf.keras.callbacks.TensorBoard(run_logdir)
    if opt == 'momentum':
        model = build_model(optimizer=opt, momentum=0.5)
    else:
        model = build_model(optimizer=opt)
    model.fit(X_train, y_train, epochs=100, callbacks=[tensorboard_cb, es])
    params = model.evaluate(X_test, y_test)
    opt_list.append((opt, params[0], params[1]))
print(opt_list)

Epoch 1/100
13/13 [==============================] - 0s 1ms/step - loss: 83732.3750 - mae: 99.9468
Epoch 2/100
13/13 [==============================] - 0s 1ms/step - loss: 122.9807 - mae: 8.1737
Epoch 3/100
13/13 [==============================] - 0s 1ms/step - loss: 93.0585 - mae: 7.1528
Epoch 4/100
13/13 [==============================] - 0s 1ms/step - loss: 82.7675 - mae: 6.6219
Epoch 5/100
13/13 [==============================] - 0s 1ms/step - loss: 76.3989 - mae: 6.4053
Epoch 6/100
13/13 [==============================] - 0s 1ms/step - loss: 73.2667 - mae: 6.1773
Epoch 7/100
13/13 [==============================] - 0s 668us/step - loss: 71.5863 - mae: 5.9187
Epoch 8/100
13/13 [==============================] - 0s 667us/step - loss: 68.0666 - mae: 5.8083
Epoch 9/100
13/13 [==============================] - 0s 1ms/step - loss: 66.3178 - mae: 5.6422
Epoch 10/100
13/13 [==============================] - 0s 1ms/step - loss: 67.8043 - mae: 5.6757
Epoch 11/100
13/13 [====================

In [68]:
open_file = open("opt.pkl", "wb")
pickle.dump(opt_list, open_file)
open_file.close()

In [69]:
open_file = open("opt.pkl", "rb")
loaded_list = pickle.load(open_file)
open_file.close()
print(loaded_list)

[('sgd', 70.1563949584961, 6.567811965942383), ('adam', 12832.2255859375, 110.19389343261719), ('nesterov', 61.545066833496094, 6.115115165710449), ('momentum', 91.13220977783203, 7.657899379730225)]


In [70]:
tf.keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [71]:
mom_list = []
momentums = [0.1, 0.5, 0.9]
for mom in momentums:
    run_logdir = get_run_logdir('_mom_', mom)
    tensorboard_cb = tf.keras.callbacks.TensorBoard(run_logdir)
    model = build_model(optimizer='momentum', momentum=mom)
    model.fit(X_train, y_train, epochs=100, callbacks=[tensorboard_cb, es])
    params = model.evaluate(X_test, y_test)
    mom_list.append((mom, params[0], params[1]))
print(mom_list)

Epoch 1/100
13/13 [==============================] - 0s 1ms/step - loss: 83728.2344 - mae: 99.7617
Epoch 2/100
13/13 [==============================] - 0s 1ms/step - loss: 111.8574 - mae: 7.7539
Epoch 3/100
13/13 [==============================] - 0s 2ms/step - loss: 92.5026 - mae: 7.1335
Epoch 4/100
13/13 [==============================] - 0s 1ms/step - loss: 86.4758 - mae: 6.8017
Epoch 5/100
13/13 [==============================] - 0s 1ms/step - loss: 82.8550 - mae: 6.7412
Epoch 6/100
13/13 [==============================] - 0s 1ms/step - loss: 81.9884 - mae: 6.6559
Epoch 7/100
13/13 [==============================] - 0s 1ms/step - loss: 82.4515 - mae: 6.5522
Epoch 8/100
13/13 [==============================] - 0s 667us/step - loss: 80.9828 - mae: 6.6153
Epoch 9/100
13/13 [==============================] - 0s 1ms/step - loss: 80.6428 - mae: 6.5149
Epoch 10/100
13/13 [==============================] - 0s 1ms/step - loss: 82.5981 - mae: 6.5743
Epoch 11/100
13/13 [======================

In [72]:
open_file = open("mom.pkl", "wb")
pickle.dump(mom_list, open_file)
open_file.close()

In [73]:
open_file = open("mom.pkl", "rb")
loaded_list = pickle.load(open_file)
open_file.close()
print(loaded_list)

[(0.1, 82.3486328125, 7.22255802154541), (0.5, 70.70952606201172, 6.956175327301025), (0.9, 62.3004035949707, 5.834388732910156)]


In [74]:
param_distribs = {
    "model__n_hidden": [0, 1, 2, 3],
    "model__n_neurons": [5, 25, 125],
    "model__learning_rate": [0.000001, 0.00001, 0.0001],
    "model__optimizer": ['sgd', 'adam', 'nesterov', 'momentum'],
    "model__momentum": [0.1, 0.5, 0.9]
}

In [75]:
import scikeras
from scikeras.wrappers import KerasRegressor
es = tf.keras.callbacks.EarlyStopping(patience=10, min_delta=1.0, verbose=1)
keras_reg = KerasRegressor(build_model, callbacks=[es])

In [76]:
from sklearn.model_selection import RandomizedSearchCV
rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=30, cv=3, verbose=2)
rnd_search_cv.fit(X_train, y_train, epochs=100, validation_split=0.1)
rnd_search_cv.best_params_

Fitting 3 folds for each of 30 candidates, totalling 90 fits
Epoch 1/100
8/8 [==============================] - 1s 26ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 2/100
8/8 [==============================] - 0s 5ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 3/100
8/8 [==============================] - 0s 3ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 4/100
8/8 [==============================] - 0s 5ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 5/100
8/8 [==============================] - 0s 5ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 6/100
8/8 [==============================] - 0s 5ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 7/100
8/8 [==============================] - 0s 3ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 8/100
8/8 [==============================] - 0s 5ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Ep

C:\Users\piotr\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\piotr\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\piotr\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 429, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "C:\Users\piotr\anaconda3\lib\site-packages\scikeras\wrappers.py", line 1100, in score
    return self.scorer(y, y_pred, sample_weight=sample_weight, **score_args)
  File "C:\Users\piotr\anaconda3\lib\site-packages\scikeras\wrappers.py", line 1697, in scorer
    return sklearn_r2_score(y_true, y_pred, **kwargs)
  File "C:\Users\piotr\anaconda3\lib\site-packages\sklearn\metrics\_regression.py", line 911, in r2_score
    y_type, y_tru

8/8 [==============================] - 1s 27ms/step - loss: 7657043506052988928.0000 - mae: 1007819584.0000 - val_loss: 2308499570688.0000 - val_mae: 1519374.6250
Epoch 2/100
8/8 [==============================] - 0s 3ms/step - loss: 2305448214528.0000 - mae: 1518370.1250 - val_loss: 2301122838528.0000 - val_mae: 1516945.1250
Epoch 3/100
8/8 [==============================] - 0s 3ms/step - loss: 2298081443840.0000 - mae: 1515942.2500 - val_loss: 2293769961472.0000 - val_mae: 1514519.7500
Epoch 4/100
8/8 [==============================] - 0s 5ms/step - loss: 2290738528256.0000 - mae: 1513518.5000 - val_loss: 2286441201664.0000 - val_mae: 1512098.2500
Epoch 5/100
8/8 [==============================] - 0s 5ms/step - loss: 2283419467776.0000 - mae: 1511098.6250 - val_loss: 2279136034816.0000 - val_mae: 1509680.7500
Epoch 6/100
8/8 [==============================] - 0s 3ms/step - loss: 2276123475968.0000 - mae: 1508682.6250 - val_loss: 2271853412352.0000 - val_mae: 1507266.8750
Epoch 7/100


C:\Users\piotr\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\piotr\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\piotr\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 429, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "C:\Users\piotr\anaconda3\lib\site-packages\scikeras\wrappers.py", line 1100, in score
    return self.scorer(y, y_pred, sample_weight=sample_weight, **score_args)
  File "C:\Users\piotr\anaconda3\lib\site-packages\scikeras\wrappers.py", line 1697, in scorer
    return sklearn_r2_score(y_true, y_pred, **kwargs)
  File "C:\Users\piotr\anaconda3\lib\site-packages\sklearn\metrics\_regression.py", line 911, in r2_score
    y_type, y_tru

8/8 [==============================] - 0s 17ms/step - loss: 6098.3862 - mae: 61.4684 - val_loss: 68.7588 - val_mae: 6.9887
Epoch 2/100
8/8 [==============================] - 0s 4ms/step - loss: 117.8920 - mae: 7.8742 - val_loss: 37.0560 - val_mae: 5.2268
Epoch 3/100
8/8 [==============================] - 0s 7ms/step - loss: 93.2922 - mae: 6.7526 - val_loss: 37.0073 - val_mae: 5.1272
Epoch 4/100
8/8 [==============================] - 0s 8ms/step - loss: 90.2957 - mae: 6.7257 - val_loss: 37.3706 - val_mae: 5.1190
Epoch 5/100
8/8 [==============================] - 0s 8ms/step - loss: 88.9913 - mae: 6.7457 - val_loss: 37.6878 - val_mae: 5.1164
Epoch 6/100
8/8 [==============================] - 0s 8ms/step - loss: 88.3675 - mae: 6.7554 - val_loss: 36.8878 - val_mae: 5.0748
Epoch 7/100
8/8 [==============================] - 0s 8ms/step - loss: 87.5805 - mae: 6.6562 - val_loss: 36.4017 - val_mae: 5.0440
Epoch 8/100
8/8 [==============================] - 0s 8ms/step - loss: 87.0961 - mae: 6.64

...

C:\Users\piotr\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\piotr\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\piotr\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 429, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "C:\Users\piotr\anaconda3\lib\site-packages\scikeras\wrappers.py", line 1100, in score
    return self.scorer(y, y_pred, sample_weight=sample_weight, **score_args)
  File "C:\Users\piotr\anaconda3\lib\site-packages\scikeras\wrappers.py", line 1697, in scorer
    return sklearn_r2_score(y_true, y_pred, **kwargs)
  File "C:\Users\piotr\anaconda3\lib\site-packages\sklearn\metrics\_regression.py", line 911, in r2_score
    y_type, y_tru

8/8 [==============================] - 0s 17ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 2/100
8/8 [==============================] - 0s 5ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 3/100
8/8 [==============================] - 0s 3ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 4/100
8/8 [==============================] - 0s 4ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 5/100
8/8 [==============================] - 0s 3ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 6/100
8/8 [==============================] - 0s 3ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 7/100
8/8 [==============================] - 0s 3ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 8/100
8/8 [==============================] - 0s 4ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 9/100
8/8 [==============================] - 0s 4ms/step - loss: nan 

C:\Users\piotr\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\piotr\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\piotr\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 429, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "C:\Users\piotr\anaconda3\lib\site-packages\scikeras\wrappers.py", line 1100, in score
    return self.scorer(y, y_pred, sample_weight=sample_weight, **score_args)
  File "C:\Users\piotr\anaconda3\lib\site-packages\scikeras\wrappers.py", line 1697, in scorer
    return sklearn_r2_score(y_true, y_pred, **kwargs)
  File "C:\Users\piotr\anaconda3\lib\site-packages\sklearn\metrics\_regression.py", line 911, in r2_score
    y_type, y_tru

12/12 [==============================] - 0s 12ms/step - loss: 1653.8988 - mae: 26.1335 - val_loss: 59.7283 - val_mae: 5.9867
Epoch 2/100
12/12 [==============================] - 0s 2ms/step - loss: 98.3517 - mae: 7.3487 - val_loss: 46.9510 - val_mae: 5.5501
Epoch 3/100
12/12 [==============================] - 0s 4ms/step - loss: 88.8650 - mae: 6.9028 - val_loss: 38.1776 - val_mae: 5.0627
Epoch 4/100
12/12 [==============================] - 0s 3ms/step - loss: 83.3306 - mae: 6.6761 - val_loss: 32.3845 - val_mae: 4.7368
Epoch 5/100
12/12 [==============================] - 0s 3ms/step - loss: 79.7645 - mae: 6.4798 - val_loss: 30.0582 - val_mae: 4.5960
Epoch 6/100
12/12 [==============================] - 0s 2ms/step - loss: 75.3109 - mae: 6.3613 - val_loss: 30.3587 - val_mae: 4.4517
Epoch 7/100
12/12 [==============================] - 0s 2ms/step - loss: 75.8981 - mae: 6.2600 - val_loss: 27.2035 - val_mae: 4.2947
Epoch 8/100
12/12 [==============================] - 0s 3ms/step - loss: 74.3

{'model__optimizer': 'sgd',
 'model__n_neurons': 25,
 'model__n_hidden': 3,
 'model__momentum': 0.9,
 'model__learning_rate': 1e-05}

In [81]:
open_file = open("rnd_search.pkl", "wb")
pickle.dump(rnd_search_cv.best_params_, open_file)
open_file.close()

In [82]:
open_file = open("rnd_search.pkl", "rb")
loaded_list = pickle.load(open_file)
open_file.close()
print(loaded_list)

{'model__optimizer': 'sgd', 'model__n_neurons': 25, 'model__n_hidden': 3, 'model__momentum': 0.9, 'model__learning_rate': 1e-05}
